In [ ]:
import numpy as np
from typing import Callable

In [ ]:
def reward(params):
    return -(np.power(params[0] - 1.0, 2) + 0.5*np.power(params[1] - 2.0, 2) + 0.25*np.power(params[2] + 1.0, 2))

In [ ]:
from evolution_strategies import es_adv_fit

params_es = es_adv_fit(reward, 3, 200, 64, 0.01, 0.1, 42)
reward_es = reward(params_es)
print(f"Advantage - Estimated params: {params_es}, reward: {reward_es}")

In [ ]:
from evolution_strategies import es_sel_succ_fit

params_es_sel = es_sel_succ_fit(reward, 3, 4*200, 0.2, 42)
reward_es_sel = reward(params_es_sel)
print(f"Selection 1/5th step size adaption - Estimated params: {params_es_sel}, reward: {reward_es_sel}")